In [1]:
import torch
import numpy as np
import random

device = 'cuda:2' if torch.cuda.is_available else 'cpu'

In [2]:
device

'cuda:2'

In [2]:
from auto_LiRPA import BoundedModule, PerturbationLpNorm, BoundedTensor
from auto_LiRPA.utils import get_spec_matrix
from cert_util import min_correct_with_eps, load_data, DeltaWrapper

from model_defs import mnist_cnn_4layer,mnist_conv_small,mnist_conv_big

/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [16]:
import sys

In [22]:
sys.path.append('/home/debangshu/uap-robustness/')

In [23]:
import src.util as util

In [19]:
sys.path

['/home/debangshu/uap-robustness/baseline/Universal_Pert_Cert',
 '/opt/anaconda/envs/nn_verify/lib/python310.zip',
 '/opt/anaconda/envs/nn_verify/lib/python3.10',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/lib-dynload',
 '',
 '/home/debangshu/.local/lib/python3.10/site-packages',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/auto_LiRPA-0.3.1-py3.10.egg',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/torch-1.12.1-py3.10-linux-x86_64.egg',
 '/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/home/debangshu/uap-robustness/']

In [3]:
def bounded_results(new_image, eps, C, bounded_model):
    ptb = PerturbationLpNorm(norm = np.inf, eps = eps)
    # bounded_delta = BoundedTensor(delta, ptb)
    bounded_images = BoundedTensor(new_image, ptb)
    final_name = bounded_model.final_name
    input_name = '/input.1' 

    result = bounded_model.compute_bounds(
        x=(bounded_images,), method='CROWN-Optimized', C=C,
        return_A=True, 
        needed_A_dict={ final_name: [input_name] },
    )
    lower, upper, A_dict = result
    lA = A_dict[final_name][input_name]['lA']
    uA = A_dict[final_name][input_name]['uA']

    lb = lower - ptb.concretize(delta, lA, sign=-1)
    ub = upper - ptb.concretize(delta, uA, sign=1)


    lA = torch.reshape(lA,(min(eval_num,10), num_cls-1,-1))
    return lA,lb,lower

### Reproducibility

In [4]:
my_seed = 1
torch.cuda.empty_cache()
torch.manual_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

### Hyperparameters 

In [5]:
eval_num = 30
num_cls = 10
adv_e = 8

# MNIST models, one can choose from 'mnist_conv_small', 'mnist_cnn_4layer', 'mnist_conv_big'
model_name = "mnist_conv_small" 

### Loadding the model for certification

In [6]:
net = eval(model_name)()
net.load_state_dict(torch.load('./'+model_name+'.pth'))

<All keys matched successfully>

### Loading a batch of data

In [7]:
new_image, new_label = load_data(num_imgs=eval_num, random=False, dataset='MNIST')
new_image = new_image.to(device)
C = get_spec_matrix(new_image,new_label.long(),10)

In [9]:
print(new_image.shape)
print(C.shape)

torch.Size([30, 1, 28, 28])
torch.Size([30, 9, 10])


### Model concretization

In [8]:
# eps = 0.03
# delta = torch.zeros_like(new_image[0]).unsqueeze(0)
# dummy_input = (new_image, delta)
# model = DeltaWrapper(net.to(device))
# bounded_model = BoundedModule(model, dummy_input)
# bounded_model.eval()
# final_name = bounded_model.final_name
eps = 0.06
delta = torch.zeros_like(new_image[0]).unsqueeze(0)
dummy_input = (new_image[:10])
model = net.to(device)
bounded_model = BoundedModule(model, dummy_input)
bounded_model.eval()
final_name = bounded_model.final_name

/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/auto_LiRPA-0.3.1-py3.10.egg/auto_LiRPA/parse_graph.py:147: FutureWarning: 'torch.onnx.symbolic_helper._set_opset_version' is deprecated in version 1.13 and will be removed in version 1.14. Please remove its usage and avoid setting internal variables directly.
  _set_opset_version(12)
/opt/anaconda/envs/nn_verify/lib/python3.10/site-packages/auto_LiRPA-0.3.1-py3.10.egg/auto_LiRPA/parse_graph.py:45: FutureWarning: 'torch.onnx._patch_torch._node_getitem' is deprecated in version 1.13 and will be removed in version 1.14. Please Internally use '_node_get' in symbolic_helper instead..
  attrs = {k: n[k] for k in n.attributeNames()}


### Results and comparision 

In [17]:
print(bounded_model)

BoundedModule(
  (/input.1): BoundInput(name="/input.1")
  (/1): BoundParams(name="/1")
  (/2): BoundParams(name="/2")
  (/3): BoundParams(name="/3")
  (/4): BoundParams(name="/4")
  (/5): BoundParams(name="/5")
  (/6): BoundParams(name="/6")
  (/7): BoundParams(name="/7")
  (/8): BoundParams(name="/8")
  (/input): BoundConv(name="/input")
  (/input.4): BoundRelu(name="/input.4")
  (/input.8): BoundConv(name="/input.8")
  (/12): BoundRelu(name="/12")
  (/13): BoundConstant(name="/13")
  (/14): BoundConstant(name="/14")
  (/15): BoundUnsqueeze(name="/15")
  (/16): BoundUnsqueeze(name="/16")
  (/17): BoundConcat(name="/17")
  (/18): BoundReshape(name="/18")
  (/input.12): BoundLinear(name="/input.12")
  (/20): BoundRelu(name="/20")
  (/21): BoundLinear(name="/21")
)


In [9]:
i = 0
while i < eval_num:
    print('Eps:', eps)
    new_image_temp = new_image[i:min(i+10, eval_num)]
    C_temp = C[i:min(i+10, eval_num)]
    # print(f'image temp shape {new_image_temp.shape}')
    # print(f'C temp shape {C_temp.shape}')

    alpha,beta,result = bounded_results(new_image[:10], eps, C[:10], bounded_model)
    samp_ACC = torch.sum(result.detach().cpu().min(axis=1)[0] > 0).numpy()
    samp_ACC = samp_ACC / eval_num * 100.0  
    print('Samp-wise Cert-ACC: {}%'.format(samp_ACC))

    label = new_label
    number_class = num_cls
    cert_ACC, delta = min_correct_with_eps(alpha, beta, eps, label[i:min(i+10, eval_num)], number_class=10, verbose=False)
    i += 10
    print('UP-based Cert-ACC: {}%'.format(cert_ACC / eval_num * 100.0))

Eps: 0.06
batch size 10
batch size 10
batch size 10
shape torch.Size([16, 13, 13])
sparse alpha shape torch.Size([32, 10, 2280])
Full alpha shape torch.Size([32, 10, 16, 13, 13])
shape torch.Size([32, 5, 5])
sparse alpha shape torch.Size([100, 10, 32, 5, 5])
Full alpha shape torch.Size([100, 10, 32, 5, 32, 5, 5])


RuntimeError: shape mismatch: value tensor of shape [100, 10, 32, 5, 5] cannot be broadcast to indexing result of shape [100, 10, 794, 5, 5]

In [17]:
new_image[:10].shape

torch.Size([10, 1, 28, 28])